In [2]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.neighbors import NearestNeighbors
from sklearn.externals import joblib
from tqdm import tqdm

df2 = pd.read_csv('Hot Stuff.csv')
df2 = df2[['WeekID','SongID']]
df_features = pd.read_excel('Hot 100 Audio Features.xlsx')


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
df1 = df_features
df1 = df1.drop(columns=['instrumentalness','spotify_track_preview_url','spotify_track_id'])
df2 = df2.drop_duplicates(subset='SongID')
print(df2.count())
df1.count()
df1 = df1.merge(df2).dropna()
df1 = df1.reset_index()
df1

WeekID    28474
SongID    28474
dtype: int64


,index,SongID,Performer,Song,spotify_genre,spotify_track_album,spotify_track_explicit,spotify_track_duration_ms,spotify_track_popularity,danceability,...,key,loudness,mode,speechiness,acousticness,liveness,valence,tempo,time_signature,WeekID
0,0,"AdictoTainy, Anuel AA & Ozuna","Tainy, Anuel AA & Ozuna",Adicto,['pop reggaeton'],Adicto (with Anuel AA & Ozuna),0.0,270740.0,91.0,0.734,...,10.0,-4.803,0.0,0.0735,0.017000,0.1790,0.623,80.002,4.0,10/5/2019
1,2,ShallowLady Gaga & Bradley Cooper,Lady Gaga & Bradley Cooper,Shallow,"['dance pop', 'pop']",A Star Is Born Soundtrack,0.0,215733.0,88.0,0.572,...,7.0,-6.362,1.0,0.0308,0.371000,0.2310,0.323,95.799,4.0,10/13/2018
2,3,ShallowLady Gaga & Bradley Cooper,Lady Gaga & Bradley Cooper,Shallow,"['dance pop', 'pop']",A Star Is Born Soundtrack,0.0,215733.0,89.0,0.572,...,7.0,-6.362,1.0,0.0308,0.371000,0.2310,0.323,95.799,4.0,10/13/2018
3,4,EnemiesPost Malone Featuring DaBaby,Post Malone Featuring DaBaby,Enemies,"['dfw rap', 'melodic rap', 'rap']",Hollywood's Bleeding,1.0,196760.0,86.0,0.542,...,6.0,-4.169,1.0,0.2100,0.058800,0.0955,0.667,76.388,4.0,9/21/2019
4,5,"Bacc At It AgainYella Beezy, Gucci Mane & Quavo","Yella Beezy, Gucci Mane & Quavo",Bacc At It Again,"['dfw rap', 'rap', 'southern hip hop', 'trap']",Bacc At It Again,1.0,228185.0,61.0,0.948,...,8.0,-5.725,0.0,0.1680,0.001240,0.0716,0.856,135.979,4.0,6/8/2019
5,6,"Bacc At It AgainYella Beezy, Gucci Mane & Quavo","Yella Beezy, Gucci Mane & Quavo",Bacc At It Again,"['dfw rap', 'rap', 'southern hip hop', 'trap']",Bacc At It Again,1.0,228185.0,62.0,0.948,...,8.0,-5.725,0.0,0.1680,0.001240,0.0716,0.856,135.979,4.0,6/8/2019
6,7,The ArcherTaylor Swift,Taylor Swift,The Archer,"['dance pop', 'pop', 'post-teen pop']",Lover,0.0,211240.0,76.0,0.292,...,0.0,-9.375,1.0,0.0401,0.120000,0.0663,0.166,124.344,4.0,8/3/2019
7,8,VIBEZDaBaby,DaBaby,VIBEZ,"['nc hip hop', 'rap']",KIRK,1.0,144935.0,88.0,0.768,...,1.0,-2.708,0.0,0.3070,0.113000,0.1070,0.777,154.187,4.0,10/12/2019
8,9,When I Grow UpNF,NF,When I Grow Up,"['hip hop', 'pop rap', 'rap']",When I Grow Up,0.0,196800.0,74.0,0.817,...,2.0,-3.985,1.0,0.1850,0.320000,0.1520,0.247,109.994,4.0,7/13/2019
9,10,Hollywood's BleedingPost Malone,Post Malone,Hollywood's Bleeding,"['dfw rap', 'melodic rap', 'rap']",Hollywood's Bleeding,0.0,156266.0,85.0,0.405,...,4.0,-3.206,0.0,0.0476,0.331000,0.1030,0.170,130.218,4.0,9/21/2019


In [5]:
# Cleaning out the top Performer
df_artist = pd.DataFrame(columns=['artist'])

for i in tqdm(range(df1.shape[0])):
    s = df1.loc[i,'Performer']
    s = s.replace(' &',',')
    s = s.replace(' Featuring',',')
    s = s.rsplit(', ')
    df_artist = df_artist.append({'artists':s}, ignore_index=True)
    df1.loc[i,'Performer'] = s[0]    

100%|██████████| 23558/23558 [03:18<00:00, 118.70it/s]


In [9]:
df_artist['artists']
df1 = df1.drop(columns='index')



KeyError: "['index'] not found in axis"

In [7]:
to_normalize = ['key','tempo','loudness']

for i in to_normalize:
    try:
        for j in tqdm(range(0,df1.shape[0])):
            df1.loc[j,i] = (df1.loc[j,i] - df1[i].min())/(df1[i].max() - df1[i].min())
    except:
        pass

100%|██████████| 23558/23558 [00:40<00:00, 576.62it/s]


In [8]:
df1.to_json('SongData.json')

In [25]:
df1
df1 = df1.drop_duplicates(subset='SongID')
df1 = df1.reset_index()
df1 = df1.drop(columns='index')
df1.to_csv('SongData.csv')